In [3]:
import gradio as gr
from transformers import pipeline
import pandas as pd
import numpy as np
from pynput import keyboard
import time
import json
from pynput.keyboard import Key, Controller, KeyCode

In [5]:

pipe = pipeline(task="zero-shot-audio-classification", model="laion/clap-htsat-fused")

In [6]:
# Load the macros from the JSON file
with open('macros.json', 'r') as file:
    data = json.load(file)

# Extract macros from the loaded data
macros = data["macros"]

# Creates a controller
board = Controller()

# Create a dictionary to store macros
macro_functions = {}

# Iterates over every macro
for macro in macros:
    # Macro details
    name = macro["name"]
    keycodes = macro["keycodes"]
    delay = macro["delay"]

# Defines the macro function
    macro_function = lambda keycodes=keycodes, delay=delay: [
        # Press the key according to the keycode
        (board.press(Key.enter) if keycode == Key.enter else
        board.press(Key.space) if keycode == Key.space else
        # If keycode is a single character then press that key
        board.press(keycode) if len(keycode) == 1 else
        # Else if the key is in the form <...> then press that key
        board.press(KeyCode.from_vk(int(keycode[1:-1]))),
        # Delay
        time.sleep(delay),
        # Release the key according to the key code
        board.release(Key.enter) if keycode == Key.enter else
        board.release(Key.space) if keycode == Key.space else
        # If keycode is a single character then release that key
        board.release(keycode) if len(keycode) == 1 else
        # Else if the key is in the form <...> then release that key
        board.release(KeyCode.from_vk(int(keycode[1:-1]))))
        # Iterate over every keycode in the macro
        for keycode in keycodes
    ]

    # Add the macro function to the dictionary with the macro name as the key
    macro_functions[name] = macro_function
    

In [11]:
#empty array 
audio_array = np.empty([])

word_detected = False


actions = []
scores = [0.998,0.01,0.22]
for x in macro_functions.keys():
    actions.append(x)

#actions = ["nothing","clap","snap",]


#CLAP model's prediction function
def classify(audio):
    global audio_array, word_detected, actions
    i = 0
    
    samplerate, array = audio
    
    #converts the input live audio array to numpy array and appends it to current audio array
    live_array = np.array(array)
    audio_array = np.append(audio_array, live_array)
    
    dimension = audio_array.shape
    
    
    #removes 5 second of audio from array if it's longer than 5 seconds
    if((dimension[0]) > (240000)):
        audio_array = audio_array[-240000:]
        print("Audio size: ", audio_array.shape)
    
    #Pass audio to model for predictions
    result = pipe(audio_array, candidate_labels=actions)
    
    #Formats the result to a dataframe and retrives the highest scoring label
    df = pd.DataFrame(result)
    max = df.idxmax(numeric_only=True)
    index = int(max[0])
    word = df.iat[index,1]
    print(df)
    print(word)
    
    #Checks if the highest scored label was an action label
    if word != "nothing":
        word_detected = True
        print("word was detected: ", word)
        print("pressing macros...")
        macro_functions[word]()
        
   
    #if word is detected or if the audio reaches 5 seconds, 
    #then reset the audio array to empty
    if (word_detected == True):
        audio_array = np.empty([1,1])
        print("All contents of audio array was removed.")
        word_detected = False
    
    return df

In [29]:
#Input - list of strings, list of corresponding scores
#Output - list of dictionaries where 'score' contains score corresponding to 'label' value
def joinDictionary (labels, scores):
    combinedDictionary = []
    dictionary = {'score': 0 ,'label':'test'}
    for i in range(len(labels)):
        #Create new dictionary for each label
        dictionary['score'] = scores[i]
        dictionary['label'] = labels[i]
        #add new dictionary to combined list
        combinedDictionary.append(dictionary.copy())
    return combinedDictionary

In [30]:
result = joinDictionary(actions,scores)
result

[{'score': 0.998, 'label': 'snap'}, {'score': 0.01, 'label': 'clap'}]

In [17]:
#Settings for Gradio Interface
demo = gr.Interface(
    fn=classify,
    inputs=gr.Audio(sources=["microphone"],streaming=True),
    outputs=[gr.Dataframe()],
    live=True,
    
)

In [18]:
#Launches Gradio Interface
demo.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


/var/folders/16/7hh1zfjs7cq_gs48v5wmkg1r0000gn/T/ipykernel_45208/3196133890.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  index = int(max[0])


      score    label
0  0.985457  nothing
1  0.012450     clap
2  0.002093     snap
nothing


/var/folders/16/7hh1zfjs7cq_gs48v5wmkg1r0000gn/T/ipykernel_45208/3196133890.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  index = int(max[0])


      score    label
0  0.984766  nothing
1  0.007645     clap
2  0.007589     snap
nothing


...

      score    label
0  0.989532  nothing
1  0.006993     snap
2  0.003475     clap
nothing
Audio size:  (240000,)


/var/folders/16/7hh1zfjs7cq_gs48v5wmkg1r0000gn/T/ipykernel_45208/3196133890.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  index = int(max[0])


      score    label
0  0.983471  nothing
1  0.011633     snap
2  0.004896     clap
nothing
Audio size:  (240000,)


/var/folders/16/7hh1zfjs7cq_gs48v5wmkg1r0000gn/T/ipykernel_45208/3196133890.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  index = int(max[0])


      score    label
0  0.963917  nothing
1  0.020486     snap
2  0.015597     clap
nothing
